In [1]:
import random
import os
import sys
import numpy as np

In [2]:
from CGen import *
from EA_MC import *

In [3]:
import subprocess
BLENDER_PATH="C:\\Program Files\\Blender Foundation\\Blender 3.3\\blender.exe"

In [5]:
OUTPUT_PATH="E:\\Research\\Statue Generator\\Generations"

In [6]:
def gen_blender(ARR,GEN_NUM):
    
    os.mkdir(OUTPUT_PATH+"\\GEN_"+str(GEN_NUM))
    
    for i,C in enumerate(ARR):

        COLOUR=str((C[0]/255,C[1]/255,C[2]/255,C[3]))
        VERTS=C[5][0]
        FACES=C[5][1]
        RO=C[4]
        FILE_NUM=str(i)
        
        with open(f'VERTS_G{GEN_NUM}_S{i}.txt', 'w') as f:
            f.writelines(str(VERTS))
        
        with open(f'FACES_G{GEN_NUM}_S{i}.txt', 'w') as f:
            f.writelines(str(FACES))

        subprocess.run(["blender", "--background", "--python","generatePopulation.py",FILE_NUM,COLOUR,f'VERTS_G{GEN_NUM}_S{i}.txt',f'FACES_G{GEN_NUM}_S{i}.txt',str(RO),str(OUTPUT_PATH+"\\GEN_"+str(GEN_NUM))],capture_output=True,shell=True)

        #print(f'Statue {i} Saved!')

## Basic Parameters for generation

In [41]:
L,B,H=6,8,10
POPULATION_SIZE=10

In [42]:
PARAMS=[]

## Initial Population

In [45]:
%%time

INITIAL_POPULATION=[]

for _ in range(POPULATION_SIZE):
    CH=generate_chromosome(L,B,H,random.random())
    INITIAL_POPULATION.append(CH)
    get_stats(CH)
    print('-'*125)

PARAMS.append(INITIAL_POPULATION)

(R,G,B,A) = (90,237,91,175)
Roughness Factor = 1
Number of vertices = 7
Number of faces = 11
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (170,254,209,172)
Roughness Factor = 4
Number of vertices = 7
Number of faces = 9
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (213,134,76,162)
Roughness Factor = 1
Number of vertices = 6
Number of faces = 9
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (117,214,230,162)
Roughness Factor = 2
Number of vertices = 7
Number of faces = 11
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (253,67,230,247)
Roughness Factor = 1
Number of vertices = 6
Number of faces = 9
----------------------------

In [46]:
%%time

os.mkdir(OUTPUT_PATH+"\\GEN_0")

for i,C in enumerate(PARAMS[0]):

    COLOUR=str((C[0]/255,C[1]/255,C[2]/255,C[3]))
    VERTS=C[5][0]
    FACES=C[5][1]
    RO=C[4]
    FILE_NUM=str(i)
    
    with open(f'VERTS_G{0}_S{i}.txt', 'w') as f:
        f.writelines(str(VERTS))
        
    with open(f'FACES_G{0}_S{i}.txt', 'w') as f:
        f.writelines(str(FACES))

    subprocess.run(["blender", "--background", "--python","generatePopulation.py",FILE_NUM,COLOUR,str(VERTS),str(FACES),str(RO),str(OUTPUT_PATH+"\\GEN_0")],capture_output=True,shell=True)
    
    print(f'Statue {i} Saved!')
    

Statue 0 Saved!
Statue 1 Saved!
Statue 2 Saved!
Statue 3 Saved!
Statue 4 Saved!
Statue 5 Saved!
Statue 6 Saved!
Statue 7 Saved!
Statue 8 Saved!
Statue 9 Saved!
Wall time: 42.2 s


## Crossover and Mutation for more generations

In [48]:
GENERATIONS=50

In [49]:
%%time

BEST_FITNESS=[]
GEN_NUM=1

for _ in range(GENERATIONS):
    
    PATH_FOR_PREV_GEN=OUTPUT_PATH+"\\GEN_"+str(GEN_NUM-1)
    FITNESS=fitness(PATH_FOR_PREV_GEN)
#     print(FITNESS)
    
    BEST_FITNESS.append(np.argmax(FITNESS))
    print(f'Statue-{np.argmax(FITNESS)} has the best fitness value in {GEN_NUM-1} generation.')
    
    MODELS=[]
    
    for _ in range(int(POPULATION_SIZE/2)):
    
        #selecting 2 random parents from previous generation
        idx1,idx2=random.randint(0,POPULATION_SIZE-1),random.randint(0,POPULATION_SIZE-1)
        p1=idx1 if FITNESS[idx1]>FITNESS[idx2] else idx2
        
        idx3,idx4=random.randint(0,POPULATION_SIZE-1),random.randint(0,POPULATION_SIZE-1)
        p2=idx3 if FITNESS[idx3]>FITNESS[idx4] else idx4
        
        #perform crossover for the selected parents
        c1,c2=crossover(PARAMS[GEN_NUM-1][p1],PARAMS[GEN_NUM-1][p2])
        
        #mutate the children after crossover
        #m1=mutate(c1,L,B,H)
        #m2=mutate(c2,L,B,H)
        
        MODELS.append(c1)
        MODELS.append(c2)
        
    #print(len(MODELS))
    #appending the mutated childs to PARAMS array
    PARAMS.append(MODELS)
    
    #generate blender file
    gen_blender(MODELS,GEN_NUM)
    
    print(f"Generation {GEN_NUM} generated successfully!")
    print("-"*120)
    
    GEN_NUM+=1

Statue-1 has the best fitness value in 0 generation.
Generation 1 generated successfully!
------------------------------------------------------------------------------------------------------------------------
Statue-2 has the best fitness value in 1 generation.
Generation 2 generated successfully!
------------------------------------------------------------------------------------------------------------------------
Statue-5 has the best fitness value in 2 generation.


IndexError: list index out of range